<h4> 0. Imports </h4>

In [1]:
%matplotlib inline

from pathlib import Path

import numpy as np
import pandas as pd
import pandas_datareader.data as web
from scipy.stats import spearmanr

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow import keras

import matplotlib.pyplot as plt
import seaborn as sns

<h4> 1. Time series naive </h4>
Wrapper function for all models using naive time series data (predicting price)

In [2]:
def ts_arima():
    # Calculate ARIMA w/ TS data and print results / return results
    return 

In [ ]:
def ts_rnn():
    # Calculate rnn w/ TS data and print results / return results
    return 

In [ ]:
def ts_gb():
    # Calculate XGBoost w/ TS data and print results / return results
    return 

In [ ]:
def ts_rl():
    # Calculate Q learning w/ TS data and print results / return results
    return 

In [ ]:
def ts_ensemble():
    # Use ensemble of multiple models, some combination of above (NOT RL)
    #  w/ TS data and print results / return results
    return 

In [ ]:
def ts_price(model_selection):
    if model_selection == "ARIMA":
        ts_arima()
        pass
    elif model_selection == "RNN":
        ts_rnn()
        pass
    elif model_selection == "GB":
        ts_gb()
        pass
    elif model_selection == "RL":
        ts_rl()
        pass
    elif model_selection == "ENSEMBLE":
        ts_ensemble()
        pass
    else:
        pass

<h4> 2. Combining other techniques </h4>

<ol>
    <li> Incorporating NLP </li>
    Use twitter data for stocks from previous research
    <li> Use returns for evaluation </li>
    See david's notes
    <li> Predicting indicators such as RSI, Momentum </li>
    See stefan jansen's work
    <li> Non-traditional TS models </li>
    For example, clustering methods? Neural transformers? Classification models? Weighted precision and measure for metric